<a href="https://www.kaggle.com/code/lonnieqin/smoker-status-prediction-with-lgbm-and-optuna?scriptVersionId=148247965" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Smoker Status prediction with LGBM and Optuna
The goal for this competition is to predict predict a patient's smoking status given information about various other health indicators. I will use following methods:
* Model: LGBM
* CV startegy: RepeatedStratifiedKFold where n_splits=5 and n_repeats=10, 50 folds in total.
* HyperParameter tuning using Optuna on first fold.

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
import optuna
import pickle

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Load data

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s3e24/train.csv")
train.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1


In [3]:
feature_columns = list(train.columns)
feature_columns.remove("id")
target_column = "smoking"
feature_columns.remove(target_column)

## Hyperparameter Tuning

In [4]:
def hyperparameter_tuning(train_ds, valid_ds, x_val, y_val):
    best_params = dict()
    def objective(trial):
        param = {
            "objective": "binary",
            "num_class": 1,
            "metric": "AUC",
            "num_iterations": 500,
            "early_stopping_round": 100,
            "device_type": "gpu",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 2, 256),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
            "feature_pre_filter": False
        }
        # Add a callback for pruning.
        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
        model = lgb.train(param, train_ds, valid_sets=[valid_ds], callbacks=[pruning_callback])
        y_pred = model.predict(x_val)
        score = roc_auc_score(y_val, y_pred)
        if "model" not in best_params:
            best_params["model"] = model
            best_params["score"] = score
            best_params["param"] = param
        elif score > best_params["score"]:
            best_params["model"] = model
            best_params["score"] = score
            best_params["param"] = param
        return score

    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), 
        direction="maximize"
    )
    study.optimize(objective, n_trials=30)
    best_params["model_params"] = study.best_trial.params
    return best_params

## Modeling

In [5]:
models =[]
kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
for fold, (train_indices, valid_indices) in enumerate(kfold.split(train, train[target_column])):
    print("=" * 50)
    print(f"Fold {fold}")
    print("=" * 50)
    x_train = train.iloc[train_indices][feature_columns]
    y_train = train.iloc[train_indices][target_column]
    x_val = train.iloc[valid_indices][feature_columns]
    y_val = train.iloc[valid_indices][target_column]
    train_ds = lgb.Dataset(x_train, label=y_train)
    valid_ds = lgb.Dataset(x_val, label=y_val)
    if fold == 0:
        best_params = hyperparameter_tuning(train_ds, valid_ds, x_val, y_val)
    param = best_params["param"]
    param["num_iterations"] = 1000
    model =  lgb.train(param, train_ds, valid_sets=[valid_ds])
    models.append(model)
    

[I 2023-10-27 16:44:32,428] A new study created in memory with name: no-name-ec117f6a-0678-4a22-bf68-1a613807fc05


Fold 0


/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2023-10-27 16:44:57,569] Trial 0 finished with value: 0.859253153233548 and parameters: {'lambda_l1': 1.1155002991081973e-07, 'learning_rate': 0.003127094961524365, 'lambda_l2': 4.043250018339267e-08, 'num_leaves': 203, 'feature_fraction': 0.48260983511738376, 'bagging_fraction': 0.7724600502982879, 'bagging_freq': 6, 'min_child_samples': 14}. Best is trial 0 with value: 0.859253153233548.
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2023-10-27 16:45:13,871] Trial 1 finished with value: 0.8656071554619853 and parameters: {'lambda_l1': 0.0022552402894474802, 'learning_rate': 0.00860392310926

[1]	valid_0's auc: 0.83252
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.840862
[3]	valid_0's auc: 0.84187
[4]	valid_0's auc: 0.845689
[5]	valid_0's auc: 0.848163
[6]	valid_0's auc: 0.850921
[7]	valid_0's auc: 0.851363
[8]	valid_0's auc: 0.851642
[9]	valid_0's auc: 0.853319
[10]	valid_0's auc: 0.853328
[11]	valid_0's auc: 0.853403
[12]	valid_0's auc: 0.853432
[13]	valid_0's auc: 0.853842
[14]	valid_0's auc: 0.853756
[15]	valid_0's auc: 0.854147
[16]	valid_0's auc: 0.854632
[17]	valid_0's auc: 0.854996
[18]	valid_0's auc: 0.854526
[19]	valid_0's auc: 0.854351
[20]	valid_0's auc: 0.854646
[21]	valid_0's auc: 0.854925
[22]	valid_0's auc: 0.854815
[23]	valid_0's auc: 0.854945
[24]	valid_0's auc: 0.85509
[25]	valid_0's auc: 0.855358
[26]	valid_0's auc: 0.855641
[27]	valid_0's auc: 0.855811
[28]	valid_0's auc: 0.855901
[29]	valid_0's auc: 0.855949
[30]	valid_0's auc: 0.856073
[31]	valid_0's auc: 0.856254
[32]	valid_0's auc: 0.856321
[33]	valid_0's auc: 0.

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[1]	valid_0's auc: 0.83399
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.841765
[3]	valid_0's auc: 0.844087
[4]	valid_0's auc: 0.847885
[5]	valid_0's auc: 0.84981
[6]	valid_0's auc: 0.852005
[7]	valid_0's auc: 0.852518
[8]	valid_0's auc: 0.852872
[9]	valid_0's auc: 0.8543
[10]	valid_0's auc: 0.854466
[11]	valid_0's auc: 0.854868
[12]	valid_0's auc: 0.855199
[13]	valid_0's auc: 0.855584
[14]	valid_0's auc: 0.855578
[15]	valid_0's auc: 0.855717
[16]	valid_0's auc: 0.855997
[17]	valid_0's auc: 0.856287
[18]	valid_0's auc: 0.855978
[19]	valid_0's auc: 0.855988
[20]	valid_0's auc: 0.856265
[21]	valid_0's auc: 0.856595
[22]	valid_0's auc: 0.856641
[23]	valid_0's auc: 0.856712
[24]	valid_0's auc: 0.856957
[25]	valid_0's auc: 0.857237
[26]	valid_0's auc: 0.85741
[27]	valid_0's auc: 0.85752
[28]	valid_0's auc: 0.857461
[29]	valid_0's auc: 0.857564
[30]	valid_0's auc: 0.857629
[31]	valid_0's auc: 0.857809
[32]	valid_0's auc: 0.857816
[33]	valid_0's auc: 0.858

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[1]	valid_0's auc: 0.832365
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.839004
[3]	valid_0's auc: 0.841646
[4]	valid_0's auc: 0.845852
[5]	valid_0's auc: 0.847513
[6]	valid_0's auc: 0.849732
[7]	valid_0's auc: 0.8505
[8]	valid_0's auc: 0.850699
[9]	valid_0's auc: 0.852146
[10]	valid_0's auc: 0.852299
[11]	valid_0's auc: 0.852753
[12]	valid_0's auc: 0.852953
[13]	valid_0's auc: 0.853555
[14]	valid_0's auc: 0.853605
[15]	valid_0's auc: 0.853782
[16]	valid_0's auc: 0.854153
[17]	valid_0's auc: 0.854207
[18]	valid_0's auc: 0.853825
[19]	valid_0's auc: 0.853877
[20]	valid_0's auc: 0.85411
[21]	valid_0's auc: 0.854392
[22]	valid_0's auc: 0.854286
[23]	valid_0's auc: 0.854281
[24]	valid_0's auc: 0.854348
[25]	valid_0's auc: 0.854745
[26]	valid_0's auc: 0.854901
[27]	valid_0's auc: 0.855008
[28]	valid_0's auc: 0.854958
[29]	valid_0's auc: 0.855042
[30]	valid_0's auc: 0.855127
[31]	valid_0's auc: 0.855209
[32]	valid_0's auc: 0.855279
[33]	valid_0's auc: 0.

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[1]	valid_0's auc: 0.832686
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.839157
[3]	valid_0's auc: 0.841063
[4]	valid_0's auc: 0.845213
[5]	valid_0's auc: 0.847565
[6]	valid_0's auc: 0.850187
[7]	valid_0's auc: 0.850822
[8]	valid_0's auc: 0.850912
[9]	valid_0's auc: 0.852473
[10]	valid_0's auc: 0.852653
[11]	valid_0's auc: 0.852799
[12]	valid_0's auc: 0.852853
[13]	valid_0's auc: 0.853155
[14]	valid_0's auc: 0.853066
[15]	valid_0's auc: 0.853294
[16]	valid_0's auc: 0.85394
[17]	valid_0's auc: 0.854283
[18]	valid_0's auc: 0.853705
[19]	valid_0's auc: 0.853538
[20]	valid_0's auc: 0.853983
[21]	valid_0's auc: 0.85416
[22]	valid_0's auc: 0.854116
[23]	valid_0's auc: 0.854053
[24]	valid_0's auc: 0.85425
[25]	valid_0's auc: 0.854453
[26]	valid_0's auc: 0.85469
[27]	valid_0's auc: 0.854677
[28]	valid_0's auc: 0.854678
[29]	valid_0's auc: 0.854822
[30]	valid_0's auc: 0.854852
[31]	valid_0's auc: 0.855052
[32]	valid_0's auc: 0.855117
[33]	valid_0's auc: 0.8

## Create submission

In [6]:
test = pd.read_csv("/kaggle/input/playground-series-s3e24/test.csv")
test[target_column] = np.median([model.predict(test[feature_columns]) for model in models], axis=0)
test[["id", target_column]].to_csv("submission.csv", index=False)
test[["id", target_column]].head()

,id,smoking
0,159256,0.660622
1,159257,0.243960
2,159258,0.430238
3,159259,0.012360
4,159260,0.596764
